In [1]:
import firebase_admin
from firebase_admin import credentials, firestore
import json
import pandas as pd

# Initialize Firebase
cred = credentials.Certificate("/home/poiqwepoi/octo_secrets/octogone-2024-fs.json")
firebase_admin.initialize_app(cred)

# Get Firestore client
db = firestore.client()


In [ ]:
collections = db.collections()
for collection in collections:
    print(collection.id)



# Show sample of client documents

In [ ]:
# Get a reference to the 'clients' collection
clients_ref = db.collection("clients")

# Fetch up to 5 documents (adjust as needed)
docs = clients_ref.limit(5).stream()

# Print each document
for doc in docs:
    print(f"Document ID: {doc.id}")
    print(doc.to_dict())  # Convert Firestore document to dictionary
    print("-" * 40)  # Separator for readability


In [ ]:
query = db.collection("clients").stream()

for doc in query:
    data = doc.to_dict()
    if "name" in data and "Rioux" in data["name"]:
        print(f"Document ID: {doc.id}")
        print(data)
        print("-" * 40)


In [ ]:
# find in product collection product with document ID 01ed23998d6dccc6acf86ab638f9c5c1
product = db.collection('clients').document('chezRioux').get()
print(product.to_dict())

In [ ]:
doc_ref = db.collection("clients").document("chezRioux")

# List subcollections
subcollections = doc_ref.collections()

for subcollection in subcollections:
    print(subcollection.id)  # Print subcollection names



In [ ]:
subcollection_ref = db.collection("clients").document("chezRioux").collection("establishments")
docs = subcollection_ref.stream()

for doc in docs:
    print(f"Document ID: {doc.id}")
    print(doc.to_dict())  # Convert Firestore document to dictionary
    print("-" * 40)


In [ ]:
global_doc_ref = db.collection("clients").document("chezRioux").collection("establishments").document("global")

# List subcollections
subcollections = global_doc_ref.collections()

for subcollection in subcollections:
    print(subcollection.id)  # Print subcollection names



In [ ]:

products_ref = (
    db.collection("clients")
    .document("chezRioux")
    .collection("establishments")
    .document("global")
    .collection("products")
)

docs = products_ref.stream()
for doc in docs:
    myprod = doc.to_dict()
    print(myprod)
    myingredientinfo = myprod["ingredientInfo"]
    print(myingredientinfo)
    volume_equivalence = myingredientinfo["volumeEquivalence"]
    print("volEqui: ", volume_equivalence)
    unitequivalence = myingredientinfo["unitEquivalence"]
    print("unitEqui: ", unitequivalence)
    weightequivalence = myingredientinfo["weightEquivalence"]
    print("weightEqui: ",  weightequivalence)
    break



In [ ]:
import json
import pandas as pd

products_ref = (
    db.collection("clients")
    .document("chezRioux")
    .collection("establishments")
    .document("global")
    .collection("products")
)

docs = products_ref.stream()

"""
for doc in docs:
    print(f"Document ID: {doc.id}")
    print(doc.to_dict())  # Convert Firestore document to dictionary
    print("-" * 40)
"""
data =[]

for doc in docs:
    myprod = doc.to_dict()

    # Extract required fields
    product_id = doc.id
    product_name = myprod.get("name", None)
    text_search = myprod.get("textSearch", None)
    product_suppliers = myprod.get("productSuppliers", None)
    
    # Extract latestInvoiceLine details
    latest_invoice = myprod.get("latestInvoiceLine", {})

    if latest_invoice:
        # Extract pricePerUnit directly
        price_per_unit = latest_invoice.get("pricePerUnit", None)

        # Extract packaging and price from supplierInvoiceLineInfo
        supplier_info = latest_invoice.get("supplierInvoiceLineInfo", "{}")
        supplier_info = json.loads(supplier_info) if isinstance(supplier_info, str) else supplier_info

        packaging = supplier_info.get("packaging", None)
        price = supplier_info.get("price", None)
    else:
        price_per_unit = None
        packaging = None
        price = None

    # Append structured data to list
    data.append({
        "id": product_id,
        "name": product_name,
        "textSearch": text_search,
        "productSuppliers": product_suppliers,
        "packaging": packaging,
        "price": price,
        "pricePerUnit": price_per_unit,
    })

# Convert to DataFrame
products_df = pd.DataFrame(data)

# Display the DataFrame
print(products_df)



In [ ]:
print(products_df[["id", "name", "packaging","pricePerUnit"]])
products_df.to_csv("products.csv", index=False)

In [ ]:
"""myprod = doc.to_dict()

myprod.keys()
#print(myprod.productSuppliers, " ", myprod.name, " " , myprod.textSearch)


# print values for keys productSuppliers, name and textSearch
print(myprod["productSuppliers"])
print(myprod["name"])
print(myprod["textSearch"])
print(myprod["id"])
print(myprod["pricePerUnit"])
"""


In [23]:
df_product_name = pd.read_csv('octogone_products_cleaned_dedup_final.csv')

In [ ]:
df_product_name.head()

In [ ]:
# Merge prod_df with df_product_name on id = product_id
merged_df = products_df.merge(df_product_name, left_on="id", right_on="product_id", how="left")

# Display merged DataFrame
print(merged_df)


In [ ]:
pd.set_option("display.max_colwidth", None)

# Show first 5 non-null productSuppliers values
print(merged_df["productSuppliers"].dropna().head(5))



In [ ]:
filtered_df = merged_df[merged_df["product_id"] == "zzuEK7hhGx3zUE5ZZ3Jq"]

# Display the row
print(filtered_df[["id", "name", "product_name", "productSuppliers"]])




In [ ]:
selected_df = filtered_df[["id", "name", "product_name", "productSuppliers"]]

# Display the new DataFrame
print(selected_df.head(10))


In [ ]:
# Filter rows where productSuppliers contains productSupplier.id = '1fU2P79RXOWhxkCmKax6'
filtered_df = merged_df[
    merged_df["productSuppliers"].apply(
        lambda suppliers: isinstance(suppliers, list) and any(
            supplier.get("id") == "1fU2P79RXOWhxkCmKax6" for supplier in suppliers
        )
    )
]

# Display the filtered DataFrame
print(filtered_df)

In [ ]:
supplier_data = []

for index, suppliers in merged_df["productSuppliers"].dropna().items():
    if isinstance(suppliers, dict):  # Ensure it's a dictionary
        for supplier_key, supplier_info in suppliers.items():
            supplier_data.append({
                "product_id": merged_df.loc[index, "id"],  # Original product ID
                "supplier_id": supplier_info.get("supplierId", None),
                "product_code": supplier_info.get("code", None),
                "priorityIndex": supplier_info.get("priorityIndex", None),
                "equivalences": supplier_info.get("equivalences", []),
            })

# Convert to DataFrame
supplier_df = pd.DataFrame(supplier_data)

# Display DataFrame
print(supplier_df)


In [ ]:
equivalence_data = []

for _, row in supplier_df.iterrows():
    for eq in row["equivalences"]:
        equivalence_data.append({
            "product_id": row["product_id"],
            "supplier_id": row["supplier_id"],
            "product_code": row["product_code"],
            "measure_id": eq.get("equivalence", {}).get("measureId", None),
            "qty_of_measure": eq.get("equivalence", {}).get("qtyOfMeasure", None),
            "abbreviation": eq.get("abbreviation", None),
        })

# Convert to DataFrame
equivalence_df = pd.DataFrame(equivalence_data)

# Display DataFrame
print(equivalence_df)


In [ ]:
import pandas as pd

# Step 1: Extract supplier-level data with product name
supplier_data = []

for index, suppliers in merged_df["productSuppliers"].dropna().items():
    if isinstance(suppliers, dict):  # Ensure it's a dictionary
        for supplier_key, supplier_info in suppliers.items():
            supplier_data.append({
                "product_id": merged_df.loc[index, "id"],  # Original product ID
                "name": merged_df.loc[index, "name"],  # Product Name
                "supplier_id": supplier_info.get("supplierId", None),
                "product_code": supplier_info.get("code", None),
                "priorityIndex": supplier_info.get("priorityIndex", None),
                "equivalences": supplier_info.get("equivalences", []),
            })

supplier_df = pd.DataFrame(supplier_data)

# Step 2: Extract equivalence-level data
equivalence_data = []

for _, row in supplier_df.iterrows():
    for eq in row["equivalences"]:
        equivalence_data.append({
            "product_id": row["product_id"],
            "name": row["name"],  # Keep product name
            "supplier_id": row["supplier_id"],
            "product_code": row["product_code"],
            "measure_id": eq.get("equivalence", {}).get("measureId", None),
            "qty_of_measure": eq.get("equivalence", {}).get("qtyOfMeasure", None),
            "abbreviation": eq.get("abbreviation", None),
        })

equivalence_df = pd.DataFrame(equivalence_data)

# Step 3: Merge supplier_df and equivalence_df
final_df = supplier_df.drop(columns=["equivalences"]).merge(equivalence_df, on=["product_id", "name", "supplier_id", "product_code"], how="left")

# Display final DataFrame
print(final_df)


In [63]:
final_df.to_csv('final_df.csv', index=False)

In [ ]:
# Reference to suppliers collection
suppliers_ref = (
    db.collection("clients")
    .document("chezRioux")
    .collection("establishments")
    .document("global")
    .collection("suppliers")
)

# Fetch suppliers
supplier_docs = suppliers_ref.stream()

# Store supplier data in a list
supplier_data = []

for doc in supplier_docs:
    supplier_info = doc.to_dict()
    supplier_data.append({
        "supplier_id": doc.id,  # Firestore document ID
        "supplier_name": supplier_info.get("name", None),
        "supplier_email": supplier_info.get("email", None),
        "supplier_website": supplier_info.get("website", None),
    })

# Convert supplier data to DataFrame
supplier_df = pd.DataFrame(supplier_data)

# Display the supplier DataFrame
print(supplier_df)




In [ ]:
# Merge final_df with supplier_df on 'supplier_id'
final_df = final_df.merge(supplier_df, on="supplier_id", how="left")

# Display the updated DataFrame
print(final_df)


In [ ]:
# Export final_df to an Excel file
final_df.to_excel("final_products_suppliers.xlsx", index=False)

print("Export complete: final_products_suppliers.xlsx")
